# Label The Candidates! Extract The Features!

This notebook corresponds to labeling and genearting features for each extracted candidate from the [previous notebook](1.data-loader.ipynb).

## MUST RUN AT THE START OF EVERYTHING

Load all the imports and set up the database for database operations. Plus, set up the particular candidate type this notebook is going to work with. 

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from collections import defaultdict
import csv
import os
import re


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tqdm

In [2]:
#Set up the environment
username = "danich1"
password = "snorkel"
dbname = "pubmeddb"

#Path subject to change for different os
database_str = "postgresql+psycopg2://{}:{}@/{}?host=/var/run/postgresql".format(username, password, dbname)
os.environ['SNORKELDB'] = database_str

from snorkel import SnorkelSession
session = SnorkelSession()

In [3]:
from snorkel.annotations import FeatureAnnotator, LabelAnnotator
from snorkel.features import get_span_feats
from snorkel.models import candidate_subclass
from snorkel.models import Candidate, GoldLabel
from snorkel.viewer import SentenceNgramViewer

In [4]:
edge_type = "dg"
debug = False

In [5]:
if edge_type == "dg":
    DiseaseGene = candidate_subclass('DiseaseGene', ['Disease', 'Gene'])
    edge = "disease_gene"
elif edge_type == "gg":
    GeneGene = candidate_subclass('GeneGene', ['Gene1', 'Gene2'])
    edge = "gene_gene"
elif edge_type == "cg":
    CompoundGene = candidate_subclass('CompoundGene', ['Compound', 'Gene'])
    edge = "compound_gene"
elif edge_type == "cd":
    CompoundDisease = candidate_subclass('CompoundDisease', ['Compound', 'Disease'])
    edge = "compound_disease"
else:
    print("Please pick a valid edge type")

# Look at potential Candidates

Use this to look at loaded candidates from a given set. The constants represent the index to retrieve the appropiate set. Ideally, here is where one can look at a subset of the candidate and develop label functions for candidate labeling.

In [ ]:
#dev_set = pd.read_csv("vanilla_lstm/lstm_disease_gene_holdout/train_candidates_to_ids.csv")
#dev_set.head(3)

In [ ]:
#TRAIN = 0
#DEV = 1

In [ ]:
#candidates = session.query(DiseaseGene).filter(DiseaseGene.split==DEV).offset(300).limit(100)
#candidates = session.query(DiseaseGene).filter(DiseaseGene.id.in_(dev_set["id"])).offset(400).limit(100)
#sv = SentenceNgramViewer(candidates, session)

In [ ]:
#sv

# Label Functions

Here is one of the fundamental part of this project. Below are the label functions that are used to give a candidate a label of 1,0 or -1 which corresponds to correct label, unknown label and incorrection label. The goal here is to develop functions that can label accurately label as many candidates as possible. This idea comes from the [data programming paradigm](https://papers.nips.cc/paper/6523-data-programming-creating-large-training-sets-quickly), where the goal is to be able to create labels that machine learning algorithms can use for accurate classification.  

In [20]:
if edge_type == "dg":
    from utils.disease_gene_lf import LFS
elif edge_type == "gg":
    from utils.gene_gene_lf import *
elif edge_type == "cg":
    from utils.compound_gene_lf import *
elif edge_type == "cd":
    from utils.compound_disease_lf import *
else:
    print("Please pick a valid edge type")

In [ ]:
c = session.query(DiseaseGene).filter(DiseaseGene.id.in_(target_cids)).all()
c

In [ ]:
try:
    for i, cand in enumerate(c):
        print(LFS['LF_CHECK_DISEASE_TAG'](cand))
except:
    print(i)

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

disease_desc = pd.read_table("https://raw.githubusercontent.com/dhimmel/disease-ontology/052ffcc960f5897a0575f5feff904ca84b7d2c1d/data/xrefs-prop-slim.tsv")
disease_normalization_df = pd.read_table("https://raw.githubusercontent.com/dhimmel/disease-ontology/052ffcc960f5897a0575f5feff904ca84b7d2c1d/data/slim-terms-prop.tsv")
wordnet_lemmatizer = WordNetLemmatizer()

disease_name = re.sub("\) ?", "", c[152][0].get_span())
disease_name = [wordnet_lemmatizer.lemmatize(word) for word in disease_name.split(" ")]
nltk.pos_tag(disease_name)

# Label The Candidates

Label each candidate based on the provided labels above. This code runs with realtive ease, but optimization is definitely needed when the number of label functions increases linearly.

In [12]:
from snorkel.annotations import load_gold_labels
L_gold_train = load_gold_labels(session, annotator_name='danich1', split=0)
annotated_cands_train_ids = list(map(lambda x: L_gold_train.row_index[x], L_gold_train.nonzero()[0]))

L_gold_dev = load_gold_labels(session, annotator_name='danich1', split=1)
annotated_cands_dev_ids = list(map(lambda x: L_gold_dev.row_index[x], L_gold_dev.nonzero()[0]))

In [7]:
sql = '''
SELECT id from candidate
WHERE split = 0 and type='disease_gene'
ORDER BY RANDOM()
LIMIT 10000;
'''
target_cids = [x[0] for x in session.execute(sql)]

In [8]:
target_cids

[16277577,
 24843519,
 17610496,
 1544218,
 7900758,
 26242908,
 30377765,
 1935695,
 81941,
 5186814,
 2182028,
 1072475,
 9098137,
 128499,
 19809883,
 28106684,
 26254129,
 11924383,
 31289021,
 21641693,
 22567694,
 2557295,
 12778127,
 21636140,
 28085858,
 28131628,
 20740965,
 15383420,
 5539990,
 19817688,
 11347598,
 31727331,
 27674215,
 19381122,
 9546859,
 19834408,
 22538966,
 32689765,
 13256024,
 22086998,
 3941309,
 9540578,
 16309692,
 33570865,
 20746828,
 11192602,
 33589131,
 684965,
 16293212,
 26232669,
 34943436,
 20747986,
 23045626,
 8306620,
 23946002,
 25782529,
 1688525,
 25312560,
 26239599,
 8729115,
 28558485,
 4246302,
 25778439,
 4567641,
 28582565,
 31294041,
 2272432,
 26225524,
 32664075,
 13671273,
 26268111,
 34490036,
 33570859,
 28563776,
 31746205,
 29898777,
 34051137,
 21640614,
 16280076,
 28985790,
 16735590,
 7914048,
 10359677,
 244140,
 26738476,
 15405456,
 11361190,
 21202330,
 3189016,
 3468282,
 5543629,
 8715137,
 18900502,
 1411510,

In [17]:
sql = '''
SELECT candidate_id FROM gold_label
'''
gold_cids = [x[0] for x in session.execute(sql)]
gold_cids

[24700,
 24703,
 22411,
 22426,
 22904,
 22908,
 22925,
 23375,
 81233,
 82753,
 81692,
 82372,
 81167,
 83221,
 83225,
 83229,
 83233,
 83237,
 83241,
 83422,
 81044,
 82920,
 80914,
 82846,
 127048,
 127034,
 130192,
 130195,
 130019,
 129088,
 128867,
 128592,
 128597,
 128602,
 128713,
 127106,
 130542,
 130543,
 130544,
 128005,
 129942,
 129160,
 129322,
 129328,
 127393,
 130082,
 127597,
 128377,
 130033,
 130054,
 130049,
 129984,
 126689,
 126695,
 128727,
 130815,
 130819,
 130925,
 130545,
 130927,
 130640,
 178000,
 174278,
 177603,
 175363,
 177711,
 178469,
 176072,
 178158,
 241000,
 244451,
 235144,
 231382,
 232720,
 231376,
 237268,
 236283,
 235411,
 235417,
 235510,
 235515,
 235518,
 235523,
 235529,
 235534,
 229709,
 233867,
 233873,
 231095,
 231250,
 232716,
 232731,
 232735,
 226358,
 239079,
 243774,
 243174,
 243181,
 228017,
 245558,
 480361,
 480366,
 476332,
 481835,
 481842,
 483470,
 481850,
 487622,
 478730,
 477467,
 477471,
 477477,
 477480,
 23442,

In [21]:
from  sqlalchemy.sql.expression import func
labeler = LabelAnnotator(lfs=list(LFS.values()))

cids = session.query(DiseaseGene.id).filter(DiseaseGene.id.in_(target_cids))
%time L_train = labeler.apply(split=0, cids_query=cids, parallelism=5)

cids = session.query(Candidate.id).filter(Candidate.id.in_(gold_cids))
%time L_dev = labeler.apply_existing(cids_query=cids, parallelism=5, clear=False)

#cids = session.query(Candidate.id).filter(Candidate.split==2)
#%time L_test = labeler.apply_existing(split=2, cids_query=cids, parallelism=5, clear=False)

Clearing existing...
Running UDF...
CPU times: user 16.4 s, sys: 839 ms, total: 17.2 s
Wall time: 47.9 s
Running UDF...
CPU times: user 8.62 s, sys: 288 ms, total: 8.91 s
Wall time: 17.1 s


In [19]:
L_dev.shape

(1028, 11)

In [10]:
np.savetxt('data/labeled_candidates.txt', target_cids)

In [11]:
L_train.lf_stats(session)

,j,Coverage,Overlaps,Conflicts
LF_HETNET_DISEASES,0,1.0000,1.0000,0.6936
LF_HETNET_DOAF,1,1.0000,1.0000,0.6936
LF_HETNET_DisGeNET,2,1.0000,1.0000,0.6936
LF_HETNET_GWAS,3,1.0000,1.0000,0.6936
LF_CHECK_GENE_TAG,4,0.2705,0.2705,0.1827
LF_IS_BIOMARKER,5,0.2245,0.2245,0.2244
LF_ASSOCIATION,6,0.1783,0.1783,0.1754
LF_NO_ASSOCIATION,7,0.0054,0.0054,0.0044
LF_NO_CONCLUSION,8,0.0567,0.0567,0.0379
LF_DG_DISTANCE,9,0.1206,0.1206,0.0846


# DO NOT RUN BELOW

# Generate Candidate Features

In conjunction with each candidate label, generate candidate features that will be used by some machine learning algorithms (notebook 4). This step is broken as insert takes an **incredibly** long time to run. Had to do roundabout way to load the features. **Do not run this block** and refer to the code block below. Gonna need to debug this part, when I get time.

In [ ]:
%%time
featurizer = FeatureAnnotator()
featurizer.apply(split=0, clear=False)

F_dev = featurizer.apply_existing(split=1, parallelism=5, clear=False)
F_test = featurizer.apply_existing(split=2, parallelism=5, clear=False)

# Work Around for above code

As mentioned above this code is the workaround for the broken featurizer. The intuition behind this section is to write all the generated features to a sql text file. Exploting the psql's COPY command, the time taken for inserting features drops to ~30 minutues (compared to 1 week+).

In [ ]:
%%time

group = 0
chunksize = 1e5
seen = set()
feature_key_hash = defaultdict(int)
feat_counter = 0

with open('feature_key.sql', 'wb') as f:
    with open('feature.sql', 'wb') as g:
        # Write the headers
        f.write("COPY feature_key(\"group\", name, id) from stdin with CSV DELIMITER '	' QUOTE '\"';\n")
        g.write("COPY feature(value, candidate_id, key_id) from stdin with CSV DELIMITER '	' QUOTE '\"';\n")
        
        # Set up the writers
        feature_key_writer = csv.writer(f, delimiter='\t',  quoting=csv.QUOTE_NONNUMERIC)
        feature_writer = csv.writer(g, delimiter='\t', quoting=csv.QUOTE_NONNUMERIC)
        
        # For each split get and generate features
        for split in [0,1,2]:
    
            #reset pointer to cycle through database again
            pointer = 0
            
            print(split)
            candidate_query = session.query(Candidate).filter(Candidate.split==split).limit(chunksize)
            
            while True:
                candidates = candidate_query.offset(pointer).all()
                
                if not candidates:
                    break

                for c in tqdm.tqdm(candidates):
                    try:
                        for name, value in get_span_feats(c):

                            # If the training set, set the feature hash
                            if split == 0:
                                if name not in feature_key_hash:
                                    feature_key_hash[name] = feat_counter
                                    feat_counter = feat_counter + 1
                                    feature_key_writer.writerow([group, name, feature_key_hash[name]])

                            if name in feature_key_hash:
                                # prevent duplicates from being written to the file
                                if (c.id, name) not in seen:
                                    feature_writer.writerow([value, c.id, feature_key_hash[name]])
                                    seen.add((c.id, name))

                        #To prevent memory overload
                        seen = set()
                    
                    except Exception as e:
                        print(e.message)
                        print(c)
                        print(c.get_parent().text)

                # update pointer for database
                pointer = pointer + chunksize

# Generate Coverage Stats

Before throwing our labels at a machine learning algorithm take a look at some quick stats. The code below will show the coverage and conflicts of each label function. Furthermore, this code will show the dimension of each label matrix.

In [ ]:
print(L_train.lf_stats(session, ))

In [ ]:
print(L_dev.lf_stats(session, ))